<a href="https://colab.research.google.com/github/dqj5182/Sparse_COO_Tensor_Multiplication_Pytorch/blob/main/SCTM_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**This function is based on PyTorch**

In [1]:
import torch
import numpy as np

### **Sparse Tensor (dot product) Multiplication Function** </br>
*This function allows us to do tensor matrix multiplication for two sparse coo tensors. In other words, it is like doing torch.matmul with two sparse coo tensors, outputting one sparse coo tensor. </br>*
Input: Two Sparse COO Tensors </br>
Output: One Sparse COO Tensor (dot product result of two sparse COO tensor)
*There were no sparse to dense convert

In [2]:
def sparse_coo_mul(tensor1, tensor2):
  # **tensor1 and tensor2 should be data type torch.sparse_coo_tensor
  tensor1_indices = tensor1.coalesce().indices() # Get indices of sparse coo tensor 1
  tensor2_indices = tensor2.coalesce().indices() # Get indices of sparse coo tensor 2
  tensor1_values = tensor1.coalesce().values() # Get values of sparse coo tensor 1
  tensor2_values = tensor2.coalesce().values() # Get values of sparse coo tensor 2

  def swap_two_rows(tensor):
    # This function swaps first row and second row of (usually) index tensor (first parameter for torch.sparse_coo_tensor())
    index = [1, 0]
    tensor[index,] = tensor.clone() # Swaps first and second row
    return tensor

  test_1 = tensor1_indices.t() # Transpose index tensor, as this transposed tensor has non-zero sparse tensor coordinates in each rows
  test_2 = swap_two_rows(tensor2_indices).t() # Transpose for same reason. Swapping first and second row due to "the property of mat mul"
  # "the property of mat mul"
  # If you think carefully on matrix multiplication, you can see that we are multiplying elements in rows of first matrix and cols of second matrix where
  # row index of first matrix = col index of second matrix. For such row and col, we multiply each elements of first and second matrices where col index of
  # first index = row index of second index.

  # The reason for swapping rows:
  # This is more of convenience than necessity. Let columns of first matrix as col1, columns of second matrix as col2, rows of first matrix as row1,
  # rows of second matrix as row2. As by "the property of mat mul" said above, we need to first meet row1 = col2. Then, we need to sum the multiplication
  # of elements that meet col1 = row2. This is just how matrix multiplication is. Nothing too complicated.

  mul_i_j = [] # A list that will store multiplication results of each one element from first matrix and second matrix
  # This will be later summed up just like how matrix multiplication works

  final = {} # A dict that will store output matrix coordinate and multiplication result.

  final_list = [] # A list that will store final outputted indecies and values for returning sparse coo tensor. 
  # This will be used to make returning sparse coo tensor (new_sparse_coo_tensor)

  for i, val_i in enumerate(test_1):
    for j, val_j in enumerate(test_2):

      if torch.equal(val_i[1], val_j[1]):
        mul_i_j.append([val_i[0].item(), val_j[0].item(), (tensor1_values[i]*tensor2_values[j]).item()]) # val_i[0]: row 1, val_i[1]: col 1, val_j[0]: col 2, val_j[1]: row 2

  # Try print(mul_i_j) to see how this works
  for ele1, ele2, ele3 in mul_i_j: # For each multiplications
    if (ele1, ele2) not in final: # If there were not yet multiplication result regarding the output coordinate
      final[(ele1, ele2)] = ele3 # Store the multiplication result for the regarding output coordinate
    else: # If there were already multiplication result regarding the output coordinate
      final[(ele1, ele2)] = final[(ele1, ele2)] + ele3 # Add the multiplication result for the existing regarding output coordinate

  for key, value in final.items(): # For keys and values in dictionary "final"
    temp = [key[0], key[1] ,value] # Make this dictionary as list of lists
    final_list.append(temp)

  final_list = np.array(final_list).T # Transpose back to make index tensor and value tensor used for parameters in torch.sparse_coo_tensor()

  new_index = final_list[0:2,] # Will be put into torch.sparse_coo_tensor() as first parameter 
  new_value = final_list[2,] # Will be put into torch.sparse_coo_tensor() as second parameter

  new_sparse_coo_tensor = torch.sparse_coo_tensor(new_index, new_value) # Finally made sparse_coo_tensor to be returned
  
  return new_sparse_coo_tensor

### **Testing with two sparse COO tensors** </br>

In [3]:
# Index for sparse coo tensor a
i_1 = torch.tensor([[0, 1, 1],
                  [2, 1, 2]])
# Value for sparse coo tensor a
v_1 = torch.tensor([3, 4, 5], dtype=torch.float32)

# Making sparse coo tensor a with i_1 and v_1
a = torch.sparse_coo_tensor(i_1, v_1)
a

tensor(indices=tensor([[0, 1, 1],
                       [2, 1, 2]]),
       values=tensor([3., 4., 5.]),
       size=(2, 3), nnz=3, layout=torch.sparse_coo)

In [4]:
# Index for sparse coo tensor b
i_2 = torch.tensor([[2, 1, 1],
                  [1, 0, 1]])

# Value for sparse coo tensor b
v_2 = torch.tensor([1, 7, 2], dtype=torch.float32)

# Making sparse coo tensor b with i_2 and v_2
b = torch.sparse_coo_tensor(i_2, v_2)
b

tensor(indices=tensor([[2, 1, 1],
                       [1, 0, 1]]),
       values=tensor([1., 7., 2.]),
       size=(3, 2), nnz=3, layout=torch.sparse_coo)

In [5]:
# See the result
sparse_coo_mul(a, b)

tensor(indices=tensor([[0, 1, 1],
                       [1, 0, 1]]),
       values=tensor([ 3., 28., 13.]),
       size=(2, 2), nnz=3, dtype=torch.float64, layout=torch.sparse_coo)

###**Comparing results with matmul**

1. Make sparse tensor a and b as dense tensor
2. Use torch.matmul to multiply those two tensors

In [6]:
torch.matmul(a.to_dense(), b.to_dense())

tensor([[ 0.,  3.],
        [28., 13.]])

1. Use sparse_coo_mul(a, b) to multiply two sparse tensor
2. Make the multiplication result into dense tensor for easy comparison

In [7]:
sparse_coo_mul(a, b).to_dense()

tensor([[ 0.,  3.],
        [28., 13.]], dtype=torch.float64)

Original sparse coo tensor output for reference

In [8]:
sparse_coo_mul(a, b)

tensor(indices=tensor([[0, 1, 1],
                       [1, 0, 1]]),
       values=tensor([ 3., 28., 13.]),
       size=(2, 2), nnz=3, dtype=torch.float64, layout=torch.sparse_coo)